In [1]:
import sys
sys.path.append('..')

In [2]:
from spinesUtils import *
import pandas as pd

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = make_classification(n_samples=10000, n_features=40, random_state=0)

df = pd.DataFrame(X, columns=[str(i) for i in range(X.shape[1])])

x_cols = df.columns.tolist()
df['y'] = y

train, test = train_test_split_bigdata_df(df, x_cols=x_cols, y_col='y', return_valid=False)
print(train.shape, test.shape)

(8000, 41) (2000, 41)


In [4]:
from lightgbm import LGBMClassifier

estimator = LGBMClassifier(random_state=0)
fe = TreeSequentialFeatureSelector(estimator, metrics_name='f1',
    forward=True,
    floating=True,
    log_file_path='feature_selection.log', verbose=True)

In [5]:
fe.fit(train[x_cols], train['y'])

17:12:05 2023-09-20 - metrics: f1
17:12:05 2023-09-20 - excluded columns using the variation ratio: []
17:12:05 2023-09-20 - using zero variance: []
17:12:05 2023-09-20 - The initial full model was successfully obtained.
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
17:12:07 2023-09-20 - [1/39]- feas_num 2 - # best loop,  f1 score is: 0.923984, current part of dataset best score is: 0.923984
17:12:07 2023-09-20 - [2/39]- feas_num 3 - patience: 1,  f1 score is: 0.920522, current part of dataset best score is: 0.923984
17:12:07 2023-09-20 - [3/39]- feas_num 4 - patience: 2,  f1 score is: 0.919976, current part of dataset best score is: 0.923984
17:12:07 2023-09-20 - [4/39]- feas_num 5 - patience: 3,  f1 score is: 0.918757, current part of dataset best score is: 0.923984
17:12:07 2023-09-20 - [5/39]- feas_num 6 - patience: 4,  f1 score is: 0.91866, current part of dataset best score is: 0.923984
17:12:07 2023-09-20 - [6/39]- feas_num 7 -

In [7]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [8]:
estimator = LGBMClassifier(random_state=0)
sfs = SFS(estimator, k_features='best', floating=True, n_jobs=10, scoring='f1', verbose=True)

In [12]:
from sklearn.feature_selection import RFECV, SequentialFeatureSelector
from lightgbm import LGBMClassifier

estimator = LGBMClassifier(random_state=0)
sfs = SequentialFeatureSelector(estimator, cv=5, n_jobs=10)
sfs.fit(train[x_cols], train['y'])

SequentialFeatureSelector(estimator=LGBMClassifier(random_state=0), n_jobs=10)

In [8]:
from sklearn.feature_selection import RFECV, SequentialFeatureSelector
from lightgbm import LGBMClassifier

estimator = LGBMClassifier(random_state=0)
rfecv = RFECV(estimator, step=1, cv=5, n_jobs=10, verbose=1)
rfecv.fit(train[x_cols], train['y'])

Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 fe

RFECV(cv=5, estimator=LGBMClassifier(random_state=0), n_jobs=10, verbose=1)

In [9]:
sfs.fit(train[x_cols], train['y'])

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  40 out of  40 | elapsed:    3.0s finished
Features: 1/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  39 out of  39 | elapsed:    1.2s finished
Features: 2/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  38 out of  38 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s finished
Features: 3/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  37 out of  37 | elapsed:    2.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.4s finis

SequentialFeatureSelector(estimator=LGBMClassifier(random_state=0),
                          floating=True, k_features=(1, 40), n_jobs=10,
                          scoring='f1', verbose=True)

In [10]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(sfs.transform(train[x_cols]), train['y'])

y_pred = estimator.predict(sfs.transform(test[x_cols]))

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9167072576716999


In [13]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(sfs.transform(train[x_cols]), train['y'])

y_pred = estimator.predict(sfs.transform(test[x_cols]))

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9144800777453839


In [9]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(rfecv.transform(train[x_cols]), train['y'])

y_pred = estimator.predict(rfecv.transform(test[x_cols]))

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9169499757163672


In [6]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(fe.transform(train[x_cols]), train['y'])

y_pred = estimator.predict(fe.transform(test[x_cols]))

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9107662463627546


In [26]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(train[x_cols], train['y'])

y_pred = estimator.predict(test[x_cols])

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9137847053093036


In [23]:
sfs.transform(train[x_cols]).shape, len(fe.cols)

((8000, 33), 32)

In [6]:
fe.best_cols_

[0,
 21,
 19,
 9,
 4,
 36,
 23,
 32,
 37,
 1,
 27,
 3,
 38,
 22,
 8,
 11,
 12,
 18,
 16,
 2,
 20,
 6,
 5,
 30,
 33,
 39,
 34,
 35,
 31,
 28,
 7,
 10,
 15]

In [ ]:
[0, 21, 32, 31, 10]